In [1]:
#When running on Jupyter Lab
import sys
!{sys.executable} -m pip install biopython
import random
import pandas as pd
from Bio.Seq import Seq
from nupack import *


#Illegal sequences, based on restriction enzymes
ill_seq=['GAATTC', 'TCTAGA', 'ACTAGT', 'CTGCAG', 'GCGGCCGC']


def check(x):
  for i in range(len(x)):
    if x[i]!='U' and x[i]!='T' and x[i]!='A' and x[i]!='G' and x[i]!='C':
      y='Invalid sequence, run the program again'
      break
    else:
      y=''
  return y

def gibbs(H):
	
	H=Strand(str(H), name='H')
	set1 = ComplexSet(strands=[H], complexes=SetSpec(max_size=1))

	my_model= Model(material='dna', ensemble='stacking', celsius=37)
	complex_results = complex_analysis(complexes=set1, model=my_model, compute=['pfunc'])
	
	rstr=str(complex_results)
	
	gibbs=[]
	gibbs.append(rstr.split('-')[-1])
	final=str(gibbs)
	
	return final

def rand_seq(l):
	seqlist=[]
	for i in range(l):
		seqlist.append(random.choice(nuc_list))
	seq=Seq(''.join(seqlist))
	
	return seq

nuc_list=['A','T','C','G']

#Asking for all the parameters

In=Seq(input('Give initiator sequence: \n'))
In=In.replace(' ','')


sir1=Seq(input('Give sense siR-1 sequence: \n'))
sir1=sir1.replace(' ','')
asir1=Seq(input('Give anti-sense siR-1 sequence: \n'))
asir1=asir1.replace(' ','')
sir2=Seq(input('Give sense siR-2 sequence: \n'))
sir2=sir2.replace(' ','')
asir2=Seq(input('Give anti-sense siR-2 sequence: \n'))
asir2=asir2.replace(' ','')
#tl1=int(input('Give toehold length of H1:baseistoy a* \n'))
tl1=8
#l1=int(input('Give loop length of H1: \n'))
l1=9
#l2=int(input('Give loop length for H2: /n'))
l2=8
#l3=int(input('Give loop length for H3: /n'))
l3=8

un=Seq('CTGA')
cun=un.reverse_complement()


y=check(In)
y1=check(sir1)
y2=check(sir2)
y3=check(asir1)
y4=check(asir2)


cIn=In.complement()
cIn_rev=cIn[::-1]

#For H1
#aa=a* kai bb=b*
aa=cIn_rev[0:tl1]
bb=cIn_rev[tl1:]
print(bb)

#a_rev=a kai b_rev=b
b=bb.complement()
b_rev=b[::-1]

a=aa.complement()
a_rev=a[::-1]


s1=sir1.back_transcribe()
cs1=s1.reverse_complement()
cs_1=cs1[2:]

s2=sir2.back_transcribe()
cs2=s2.reverse_complement()
cs_2=cs2[2:]

aas1=asir1.back_transcribe()
as1=aas1.reverse_complement()
as_1=as1[2:]

aas2=asir2.back_transcribe()
as2=aas2.reverse_complement()
as_2=as2[2:]

#check for illegal sequences
for i in range(len(ill_seq)):
	if ill_seq[i] in In:
		print('Illegal sequence in In, check your inputs')
		print(In)
		sys.exit()
	if ill_seq[i] in sir1:
		print('Illegal sequence in sir1, check your inputs')
		print(sir1)
		sys.exit()
	if ill_seq[i] in sir2:
		print('Illegal sequence in sir2, check your inputs')
		print(sir2)
		sys.exit()
	if ill_seq[i] in asir1:
		print('Illegal sequence in asir1, check your inputs')
		print(asir1)
		sys.exit()
	if ill_seq[i] in asir2:
		print('Illegal sequence in asir2, check your inputs')
		print(asir2)
		sys.exit()

NUM_ITS=500

df=pd.DataFrame(columns=['ID','H1','Gibbs1','loop1','H2','Gibbs2','loop2','H3','Gibbs3','loop3','H4','Gibbs4','sum_Gibbs'])
id=df['ID']

sum_Gibbs_max=0
max_i=0

for step in range(NUM_ITS):
	clist=[]
	for i in range(l1):
		clist.append(random.choice(nuc_list))
		
	c=Seq(''.join(clist))
	cc=c.complement()
	cc_rev=cc[::-1]
	
	dlist=[]
	for j in range(l2):
		dlist.append(random.choice(nuc_list))
        
	d=Seq(''.join(dlist))
	dd=d.complement()
	dd_rev=dd[::-1]
	
	elist=[]
	for k in range(l3):
		elist.append(random.choice(nuc_list))
		
	e=Seq(''.join(elist))
	ee=e.complement()
	ee_rev=ee[::-1]

	H1=aa+bb+c+b_rev+un+cs_1
	H2=cs_2+un+b_rev+d+bb+cc_rev
	H3=as_1+cun+dd_rev+bb+e+b_rev
	H4=b_rev+a_rev+bb+ee_rev+cun+as_2
	flag=True
	for i in range(len(ill_seq)):
		if ill_seq[i] in H1:
			flag=False
		if ill_seq[i] in H2:
			flag=False
		if ill_seq[i] in H3:
			flag=False
		if ill_seq[i] in H4:
			flag=False
	if flag==True:
		g1=gibbs(H1)
		g2=gibbs(H2)
		g3=gibbs(H3)
		g4=gibbs(H4)
		g1f=float(g1[2:7])
		g2f=float(g2[2:7])
		g3f=float(g3[2:7])
		g4f=float(g4[2:7])
		sum_Gibbs=g1f+g2f+g3f+g4f
		if sum_Gibbs>sum_Gibbs_max:
			sum_Gibbs_max=sum_Gibbs
			max_i=step
		df.at[step,'ID']=step
		df.at[step,'H1']=str(H1)
		df.at[step,'H2']=str(H2)
		df.at[step,'H3']=str(H3)
		df.at[step,'H4']=str(H4)
		df.at[step,'loop1']=str(c)
		df.at[step,'loop2']=str(d)
		df.at[step,'loop3']=str(e)
		df.at[step,'Gibbs1']='-'+str(g1)
		df.at[step,'Gibbs2']='-'+str(g2)
		df.at[step,'Gibbs4']='-'+str(g4)
		df.at[step,'Gibbs3']='-'+str(g3)
		df.at[step,'sum_Gibbs']='-'+str(sum_Gibbs)


df.to_csv('H sequences.csv')

df

x=df.loc[max_i,'Gibbs1']
print('H1:', df.loc[max_i,'H1'], '\n Gibbs for H1:', '-'+x[3:9], 'loop1:', df.loc[max_i,'loop1'])

y=df.loc[max_i,'Gibbs2']
print('H2:', df.loc[max_i,'H2'], '\n Gibbs for H2:', '-'+y[3:9], 'loop2:', df.loc[max_i,'loop2'])

z=df.loc[max_i,'Gibbs3']
print('H3:', df.loc[max_i,'H3'], '\n Gibbs for H3:', '-'+z[3:9], 'loop3:', df.loc[max_i,'loop3'])

w=df.loc[max_i,'Gibbs4']
print('H4:', df.loc[max_i,'H4'], '\n Gibbs for H4:', '-'+w[3:9])

df.loc[max_i,'sum_Gibbs']
print('sum_Gibbs:', df.loc[max_i,'sum_Gibbs'])

Give initiator sequence: 
 CAGACTGATGTTGAGTAGAACCGAATTTGTG
Give sense siR-1 sequence: 
 UAUUAU
Give anti-sense siR-1 sequence: 
 UAUA
Give sense siR-2 sequence: 
 AUAUA
Give anti-sense siR-2 sequence: 
 AUAA


CGGTTCTACTCAACATCAGTCTG
H1: CACAAATTCGGTTCTACTCAACATCAGTCTGCTAGTTAAGCAGACTGATGTTGAGTAGAACCGCTGAAATA 
 Gibbs for H1: -31.898 loop1: CTAGTTAAG
H2: TATCTGACAGACTGATGTTGAGTAGAACCGGCGCGAGGCGGTTCTACTCAACATCAGTCTGCTTAACTAG 
 Gibbs for H2: -32.365 loop2: GCGCGAGG
H3: TATCAGCCTCGCGCCGGTTCTACTCAACATCAGTCTGAAAAGCATCAGACTGATGTTGAGTAGAACCG 
 Gibbs for H3: -29.726 loop3: AAAAGCAT
H4: CAGACTGATGTTGAGTAGAACCGAATTTGTGCGGTTCTACTCAACATCAGTCTGATGCTTTTTCAGAT 
 Gibbs for H4: -28.285
sum_Gibbs: -122.25
